In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import collections
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow.keras import layers

In [139]:
from tqdm import tqdm

## 处理数据

一共206条路，每条路有5856个time slot数据，时间间隔是15分钟，我们用过去的12组数据，预测未来的一步

In [73]:
speed_np = np.load('/home/blu/workspace/graduate_project/STGCN/beijing_traffic/04_selected_speed_data.npy')

In [74]:
speed_df = pd.DataFrame(speed_np)

In [75]:
speed_df.columns = ['link_id', 'time', 'speed']
speed_df['time'] = speed_df['time'].astype(int)
speed_df.head()

,link_id,time,speed
0,1.561646e+09,0,27.994844
1,1.561646e+09,1,28.143859
2,1.561646e+09,2,24.732537
3,1.561646e+09,3,24.710982
4,1.561646e+09,4,31.373775


In [76]:
speed_df = speed_df.drop(['link_id'], axis=1)   #删除link_id

In [84]:
len(speed_df['time'].unique())

5856

In [85]:
df_np = []
for i in range(len(speed_df['time'].unique())):
    df_arr = speed_df[speed_df['time']==i]['speed'].values
    df_np.append(df_arr)
df_np = np.stack(df_np, axis=0)

In [87]:
df = pd.DataFrame(df_np)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
0,27.994844,30.347143,32.575597,18.727644,40.066479,37.117476,49.235025,31.165759,45.006070,36.879391,...,47.066148,38.503741,28.511387,35.717083,27.731028,33.292211,28.418867,49.235025,27.878318,39.475456
1,28.143859,31.611929,34.014326,19.190332,39.347278,39.545619,51.121558,33.794416,46.564749,38.663385,...,47.787003,40.128267,25.188556,32.000327,28.998823,33.574776,29.392379,51.121558,29.418825,42.973169
2,24.732537,26.622035,35.564099,19.013441,40.294744,42.530657,51.282347,34.538524,45.572762,39.936644,...,47.217317,43.322588,32.347406,32.610760,30.024399,33.824207,28.351918,51.282347,29.312808,43.312035
3,24.710982,25.107506,36.822186,19.108096,40.521331,43.199360,49.639349,32.813774,44.114274,40.153172,...,45.834001,44.046478,53.613536,35.440693,31.382230,32.121083,28.018398,49.639349,28.436941,40.161472
4,31.373775,28.094612,40.339869,21.048341,40.587194,43.380984,49.367218,32.259387,45.573407,38.585181,...,46.819217,43.892583,57.641994,36.095982,33.408229,32.708585,31.776648,49.367218,30.467981,37.835046


In [88]:
df.shape

(5856, 216)

In [149]:
x = []
y = []
x_offset = 12
y_offset = 1

In [151]:
index_max = (df.shape[0]) - x_offset - y_offset
for i in tqdm(range(index_max)):
    x_array = df[i : i + x_offset].values
    x_array = np.array(x_array).flatten()
    y_array = df[i + x_offset : i + x_offset + y_offset].values
    y_array = np.array(y_array).flatten()
    
    x.append(x_array.tolist())
    y.append(y_array.tolist())

x = np.stack(x, axis=0)    
y = np.stack(y, axis=0)

100%|██████████| 5843/5843 [00:01<00:00, 3218.28it/s]


In [152]:
print("x.shape = ", x.shape, " y.shape = ", y.shape)

x.shape =  (5843, 2592)  y.shape =  (5843, 216)


x与y为所有数据，按照训练集70%，验证集10%，测试集20%的比例进行划分

In [153]:
num_samples = x.shape[0]
num_test = round(num_samples * 0.2)
num_train = round(num_samples * 0.7)
num_val = num_samples - num_test - num_train

x_train, y_train = x[:num_train], y[:num_train]
x_val, y_val = (
        x[num_train: num_train + num_val],
        y[num_train: num_train + num_val],
    )
x_test, y_test = x[-num_test:], y[-num_test:]

In [156]:
np.save("/home/blu/workspace/graduate_project/STGCN/beijing_traffic/x_train.npy", x_train)
np.save("/home/blu/workspace/graduate_project/STGCN/beijing_traffic/y_train.npy", y_train)

In [157]:
np.save("/home/blu/workspace/graduate_project/STGCN/beijing_traffic/x_test.npy", x_test)
np.save("/home/blu/workspace/graduate_project/STGCN/beijing_traffic/y_test.npy", y_test)
np.save("/home/blu/workspace/graduate_project/STGCN/beijing_traffic/x_val.npy", x_val)
np.save("/home/blu/workspace/graduate_project/STGCN/beijing_traffic/y_val.npy", y_val)